In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from statistics import mode

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression

In [5]:
# Read the data
train_data = pd.read_csv('train.csv', index_col='Id')
test_data = pd.read_csv('test.csv', index_col='Id')

In [6]:
train_data.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [ ]:
#Data Cleaning
train_data.rename({'1stFlrSF': 'FirstFlrSF', '2ndFlrSF': 'SecFlrSF', '3SsnPorch': 'ThreeSsnPorch'}, axis=1, inplace=True)
test_data.rename({'1stFlrSF': 'FirstFlrSF', '2ndFlrSF': 'SecFlrSF', '3SsnPorch': 'ThreeSsnPorch'}, axis=1, inplace=True)

#X_train_clean.loc[X_train_clean['MasVnrType'] == "None"]
#drop_columns = ['Alley', 'Utilities']
#train_data.drop(drop_columns, axis=1, inplace=True)
#test_data.drop(drop_columns, axis=1, inplace=True)


train_data.loc[train_data['MSZoning'] == "C (all)", 'MSZoning'] = "C"
test_data.loc[test_data['MSZoning'] == "C (all)", 'MSZoning'] = "C"

train_data['SaleType'].fillna(value="Oth", inplace=True)
test_data['SaleType'].fillna(value="Oth", inplace=True)

train_data['Functional'].fillna(value="Typ", inplace=True)
test_data['Functional'].fillna(value="Typ", inplace=True)


for column in train_data[['MSZoning', 'Electrical', 'KitchenQual', 'Utilities']]:
    train_data[column].fillna(train_data[column].mode()[0], inplace=True)

for column in test_data[['MSZoning', 'Electrical', 'KitchenQual', 'Utilities']]:
    test_data[column].fillna(test_data[column].mode()[0], inplace=True)

train_data['LotFrontage'].fillna(train_data['LotFrontage'].mean(), inplace=True)
test_data['LotFrontage'].fillna(test_data['LotFrontage'].mean(), inplace=True)


for column in train_data[['MasVnrType', 'MiscFeature', 'Alley', 'Exterior1st', 'Exterior2nd']]:
    train_data[column].fillna(value="None", inplace=True)
for column in test_data[['MasVnrType', 'MiscFeature', 'Alley', 'Exterior1st', 'Exterior2nd']]:
    test_data[column].fillna(value="None", inplace=True)
    
for column in train_data[['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
                          'BsmtFinType2', 'GarageType', 'GarageFinish', 
                          'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'FireplaceQu', 
                         ]]:
    train_data[column].fillna(value="None", inplace=True)
    
for column in test_data[['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
                          'BsmtFinType2', 'GarageType', 'GarageFinish', 
                          'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'FireplaceQu', 
                         ]]:
    test_data[column].fillna(value="None", inplace=True)
    
for column in train_data[['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                          'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 
                          'GarageArea', 'TotalBsmtSF',
                         ]]:
    train_data[column].fillna(value=0, inplace=True)
    
for column in test_data[['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                          'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 
                          'GarageArea', 'TotalBsmtSF',
                         ]]:
    test_data[column].fillna(value=0, inplace=True)

In [ ]:
# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

# Utility functions from Tutorial
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")
    


In [ ]:
X_mi = train_data.copy()
y_mi = X_mi.pop('SalePrice')

mi_scores = make_mi_scores(X_mi, y_mi)

In [ ]:
print("Top 20 MI:")
print(mi_scores.head(20))
print("\n")
print("Bootom 20 MI:")
print(mi_scores.tail(20))  # uncomment to see bottom 20

plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores.head(20))
plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores.tail(20))  # uncomment to see bottom 20

In [ ]:
#features = ["YearBuilt", "MoSold", "ScreenPorch"]
#sns.relplot(
#    x="value", y="SalePrice", col="variable", data=df.melt(id_vars="SalePrice", value_vars=features), facet_kws=dict(sharex=False),
#);

#sns.catplot(x="BldgType", y="SalePrice", data=df, kind="boxen");

# YOUR CODE HERE: 
#feature = "GrLivArea"

#sns.lmplot(
#    x=feature, y="SalePrice", hue="BldgType", col="BldgType",
#    data=df, scatter_kws={"edgecolor": 'w'}, col_wrap=3, height=5,
#);

In [ ]:
#Drop columns with low MI score
drop_columns = [#'ExterCond',
                #'LotConfig',
                #'BsmtFullBath',
                #'Heating',
                #'BsmtHalfBath',
                #'Functional',
                #'LowQualFinSF',
                #'RoofMatl',
                #'LandSlope',
                #'YrSold',
                #'BsmtFinSF2',
                #'MiscFeature',
                #'Condition2',
                #'ThreeSsnPorch',
                'Street',
                'Utilities',
                'PoolArea',
                'PoolQC',
                'MiscVal',
                'MoSold', 
                'Alley']
train_data.drop(drop_columns, axis=1, inplace=True)
test_data.drop(drop_columns, axis=1, inplace=True)

In [ ]:
# Remove rows with missing target, separate target from predictors
# Drop rows with missing SalePrice from the Train dataset
#train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)

#Separate SalesPrice from the Train dataset
y = train_data.SalePrice
train_data.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
#X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
#                                                                train_size=0.8, test_size=0.2,
#                                                                random_state=1)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in train_data.columns if
                   #train_data[cname].nunique() < 10 and 
                   train_data[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in train_data.columns if 
                train_data[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = train_data[my_cols].copy()
#X_valid = X_valid_full[my_cols].copy()
X_test = test_data[my_cols].copy()
#X_train_clean = train_data[my_cols].copy()
#X_test_clean = test_data[my_cols].copy()

In [ ]:
print(X_test.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_test.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

X_test.describe()

In [ ]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

X_train.describe()

In [ ]:
f1_train_clean = pd.DataFrame()  # dataframe to hold new features

f1_train_clean["LivLotRatio"] = train_data.GrLivArea / train_data.LotArea
f1_train_clean["Spaciousness"] = (train_data.FirstFlrSF + train_data.SecFlrSF) / train_data.TotRmsAbvGrd
#f1_train_clean["TotalOutsideSF"] = train_data.WoodDeckSF + train_data.OpenPorchSF + train_data.EnclosedPorch + train_data.ThreeSsnPorch + train_data.ScreenPorch

print(f1_train_clean.shape)
f1_train_clean

In [ ]:
f1_test_clean = pd.DataFrame()  # dataframe to hold new features

f1_test_clean["LivLotRatio"] = test_data.GrLivArea / test_data.LotArea
f1_test_clean["Spaciousness"] = (test_data.FirstFlrSF + test_data.SecFlrSF) / test_data.TotRmsAbvGrd
#f1_test_clean["TotalOutsideSF"] = test_data.WoodDeckSF + test_data.OpenPorchSF + test_data.EnclosedPorch + test_data.ThreeSsnPorch + test_data.ScreenPorch

print(f1_test_clean.shape)
f1_test_clean

In [ ]:
f2_train_clean = pd.get_dummies(train_data.BldgType, prefix="Bldg")
# Multiply
f2_train_clean = f2_train_clean.mul(train_data.GrLivArea, axis=0)

print(f2_train_clean.shape)
f2_train_clean

In [ ]:
f2_test_clean = pd.get_dummies(test_data.BldgType, prefix="Bldg")
# Multiply
f2_test_clean = f2_test_clean.mul(test_data.GrLivArea, axis=0)

print(f2_test_clean.shape)
f2_test_clean

In [ ]:
f3_train_clean = pd.DataFrame()

# YOUR CODE HERE
porch_types = ["WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "ThreeSsnPorch", "ScreenPorch"]
f3_train_clean["PorchTypes"] = train_data[porch_types].gt(0.0).sum(axis=1)

print(f3_train_clean.shape)
f3_train_clean

In [ ]:
f3_test_clean = pd.DataFrame()

# YOUR CODE HERE
porch_types = ["WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "ThreeSsnPorch", "ScreenPorch"]
f3_test_clean["PorchTypes"] = test_data[porch_types].gt(0.0).sum(axis=1)

print(f3_test_clean.shape)
f3_test_clean

In [ ]:
f4_train_clean = pd.DataFrame()

# YOUR CODE HERE
f4_train_clean["MedNhbdArea"] =  (
    train_data.groupby("Neighborhood")  
    ["GrLivArea"]                
    .transform("median")      
)

print(f4_train_clean.shape)
f4_train_clean

In [ ]:
f4_test_clean = pd.DataFrame()

# YOUR CODE HERE
f4_test_clean["MedNhbdArea"] =  (
    test_data.groupby("Neighborhood")  
    ["GrLivArea"]                
    .transform("median")      
)

print(f4_test_clean.shape)
f4_test_clean

In [ ]:
X_train_final = train_data.join([f1_train_clean, f2_train_clean, f3_train_clean, f4_train_clean])
X_test_final = test_data.join([f1_test_clean, f2_test_clean, f3_test_clean, f4_test_clean])
#score_dataset(X_new, y)

print(X_train_final.shape)
print(X_test_final.shape)

In [ ]:
for colname in X_train_final.select_dtypes(["category", "object"]):
        X_train_final[colname], _ = X_train_final[colname].factorize()
#for colname in X_test_final.select_dtypes(["category", "object"]):
#        X_test_final[colname], _ = X_test_final[colname].factorize()

In [ ]:
X_1 = X_train_final.copy()
#X_1.drop(X_1.iloc[:, 0:72], inplace = True, axis = 1)
#X_1.drop(X_1.columns[0], axis = 1, inplace = True)
X_2 = X_test_final.copy()
#X_2.drop(X_2.iloc[:, 0:72], inplace = True, axis = 1)
#X_2.drop(X_2.columns[0], axis = 1, inplace = True)

In [ ]:
"""

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

"""

In [ ]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')# Your code here

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
]) # Your code here

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
#model = RandomForestRegressor(n_estimators=52, random_state=1) # Your code here

In [ ]:
# make sure to comment this out, before submitting 
"""
import optuna

def objective(trial):
       
    xgb_params_2 = dict(
        random_state=trial.suggest_int("random_state", 0, 3), 
        num_parallel_tree=trial.suggest_int("num_parallel_tree", 1, 3), 
        max_depth=trial.suggest_int("max_depth", 1, 10),
        learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        n_estimators=trial.suggest_int("n_estimators", 1000, 8000),
        min_child_weight=trial.suggest_int("min_child_weight", 1, 10),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.2, 1.0),
        subsample=trial.suggest_float("subsample", 0.2, 1.0),
        reg_alpha=trial.suggest_float("reg_alpha", 1e-4, 1e2, log=True),
        reg_lambda=trial.suggest_float("reg_lambda", 1e-4, 1e2, log=True),
        
    )
    xgb_2 = XGBRegressor(**xgb_params_2) #XGBRegressor
    
    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', xgb_2)
                             ])
    
    #my_pipeline.fit(X_train, y)
    #preds = my_pipeline.predict(X_test)
    #score = mean_absolute_error(y, preds) 
    #print('MAE:', score)
    #return score
   # return score_dataset(X_train, y_train, xgb)
    
    scores = -1 * cross_val_score(my_pipeline, X_train_final, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')
    
    return scores.mean()


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40)
#xgb_params_2 = study.best_params

trial = study.best_trial
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

"""

In [ ]:
# Here is the best result I got so far. 
#Accuracy: 14830.808106806508
#Best hyperparameters: {'random_state': 0, 'num_parallel_tree': 3, 
#    'max_depth': 4, 'learning_rate': 0.01032010748146519, 
#    'n_estimators': 3218, 'min_child_weight': 8, 
#    'colsample_bytree': 0.2417577000580567, 'subsample': 0.7512730499135545, 
#    'reg_alpha': 0.5274436957029306, 'reg_lambda': 0.006692716729570046}

xgb_params = dict(random_state=0,
    max_depth=4,           # maximum depth of each tree - try 2 to 10
    learning_rate=0.01032010748146519,    # effect of each tree - try 0.0001 to 0.1
    n_estimators=3218,     # number of trees (that is, boosting rounds) - try 1000 to 8000
    min_child_weight=8,    # minimum number of houses in a leaf - try 1 to 10
    colsample_bytree=0.2417577000580567,  # fraction of features (columns) per tree - try 0.2 to 1.0
    subsample=0.7512730499135545,         # 0.7 fraction of instances (rows) per tree - try 0.2 to 1.0
    reg_alpha=0.5274436957029306,         # 0.5 L1 regularization (like LASSO) - try 0.0 to 10.0
    reg_lambda=0.006692716729570046,        # L2 regularization (like Ridge) - try 0.0 to 10.0                 
    num_parallel_tree=3  # set > 1 for boosted random forests
                         ) # Your code here

my_model = XGBRegressor(**xgb_params)

In [ ]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', my_model)
                              ])

In [ ]:
# Multiply by -1 since sklearn calculates *negative* MAE
log_y = np.log(y)
scores = -1 * cross_val_score(my_model, X_1, log_y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)


In [ ]:
print("Average MAE score:", scores.mean())

#14879.336951519692 and 
#Average MAE score: 0.0800612423227955

In [ ]:
# Metric for Housing competition is RMSLE (Root Mean Squared Log Error)

score = np.sqrt(scores.mean())
print(score)

#121.91495011716945
#finalMAE is ranking 13449

#0.28295095391745106

In [ ]:
# Preprocessing of training data, fit model 
#my_pipeline.fit(X_1, y)
my_model.fit(X_1, log_y)

# Preprocessing of test data, fit model
preds_test = np.exp(my_model.predict(X_2)) # Your code here

# Preprocessing of training data, fit model 
#my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
#preds = my_pipeline.predict(X_valid)

# Evaluate the model
#score = mean_absolute_error(y, preds_test)
#print('MAE:', score)

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_2.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)